# Advanced: Understanding the `Env` lifecycle

In a previous guide, we introduced the basics of `Env` objects, which allow us to easily swich DAG configuration parameters. This guide provides a more detailed view to understand `Env` lifecycle, this is important to implement advanced patterns such as using values defined in `Env` inside hook functions.

The essential concept to understand is that there can only be a single `Env` at a given time, once it is terminated, its values are no longer available. If we try to access a value in `Env` after it has terminated, it won't work.

For `Env` basic usage this is not a problem, since values loaded from `Env` are usually simple data types such as strings or numbers that are [passed by value](https://courses.washington.edu/css342/zander/css332/passby.html), hence a copy is implicitely created. When tasks are executed, copies are used (we don't read from `Env` anymore). Here's an example:

## Basic usage: using values from env in Task or Product

In [1]:
from functools import partial
import traceback
from pathlib import Path

from ploomber import DAG, with_env, Env
from ploomber.tasks import PythonCallable
from ploomber.products import File

In [5]:
def touch(product):
    Path(str(product)).touch()
    

@with_env({'path': {'data': '~/data'}})
def make(env):
    # env is only available inside the function body, it is ended afterwards
    dag = DAG()
    # env.path.data is a string, is it passed by value to File
    PythonCallable(touch, File(env.path.data / 'file.txt'), dag=dag)
    return dag


dag = make()

# we can access (a copy) of the value
dag['touch'].product

File(/Users/Edu/data/file.txt)

## Wrong: using `Env.load()` inside a hook function

Now, say we need to send an email notification when our DAG finishes execution, and we want the recipient to be configured by the `Env` object, our first attempt is to do something like this:

In [10]:
def touch(product):
    Path(str(product)).touch()

    
def on_finish():
    env = Env.load()
    # code to send email using env.email
    print('Sending email to: %s' % env.email)
    

@with_env({'path': {'data': '~/data'}, 'email': 'someone@example.com'})
def make(env):
    dag = DAG()
    t = PythonCallable(touch, File(env.path.data / 'file.txt'),
                       dag=dag)
    dag.on_finish = on_finish
    return dag


dag = make()


try:
    dag.build()
except Exception as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-10-cba8c11673b1>", line 24, in <module>
    dag.build()
  File "/Users/Edu/dev/ploomber/src/ploomber/dag/DAG.py", line 235, in build
    report = self._build(force, show_progress)
  File "/Users/Edu/dev/ploomber/src/ploomber/dag/DAG.py", line 299, in _build
    self.on_finish(**kwargs)
  File "<ipython-input-10-cba8c11673b1>", line 6, in on_finish
    env = Env.load()
  File "/Users/Edu/dev/ploomber/src/ploomber/env/env.py", line 99, in load
    raise RuntimeError('Env has not been set, run Env() before '
RuntimeError: Env has not been set, run Env() before running Env.load()


Our code fails saying that there is no active `Env`.

## Correct: using a partial to prevent reading from `Env` when the hook is executed

The problem with the code above is that we are trying to load `Env` inside the DAG `on_finish` hook. Such function is executed when calling `DAG.build()`, by that time, the `Env` no longer exists, and the hook is unable to read the email address.

To fix this, we have to copy `env.email` so it is available to the `on_finish` hook and no longer read from `Env`, the cleanest way to do so is using [partials](https://docs.python.org/3/library/functools.html#functools.partial).

In [6]:
def touch(product):
    Path(str(product)).touch()

    
def on_finish(recipient):
    print('Sending email to: %s' % recipient)
    

@with_env({'path': {'data': '~/data'}, 'email': 'someone@example.com'})
def make(env):
    # env is only available inside the function body, it is ended afterwards
    dag = DAG()
    # env.path.data is a string, is it passed by value to File
    t = PythonCallable(touch, File(env.path.data / 'file.txt'),
                       dag=dag)
    # bind the current env.email value to a partial created from on_finish
    dag.on_finish = partial(on_finish, recipient=env.email)
    return dag


dag = make()
dag.build()


Sending email to: someone@example.com


name,Ran?,Elapsed (s),Percentage
touch,False,0,0


## Alternative (not recommended): Keep `Env` active when calling `dag.build()`

There is one way to make `Env.load()` work inside hook functions by keeping the `Env` active when calling `dag.build()`. This is not recommended as it is no longer possible to use the factory pattern appropriately.

In [4]:
def touch(product):
    Path(str(product)).touch()

    
def on_finish():
    env = Env.load()
    # code to send email using env.email
    print('Sending email to: %s' % env.email)
    

def make(env):
    # env is only available inside the function body, it is ended afterwards
    dag = DAG()
    # env.path.data is a string, is it passed by value to File
    t = PythonCallable(touch, File(env.path.data / 'file.txt'),
                       dag=dag)
    dag.on_finish = on_finish
    return dag


# keeping Env active
with Env({'path': {'data': '~/data'}, 'email': 'someone@example.com'}) as env:
    dag = make(env=env)
    dag.build()


Sending email to: someone@example.com


The problem with the previous approach is that we can no longer abstract a DAG definition in a single function call: whoever uses the DAG returned by `make()` has to know that they have to manage `Env` directly **and** that they have to keep the environment open when they call `dag.build()`.